In [1]:
import pandas as pd

In [11]:
import json
with open('types.json', 'r') as f:
    data_types = json.load(f)

In [10]:
df = pd.read_feather('featherdata.feather')

In [12]:
df = df.astype(data_types)

In [58]:
from missing_values import some_nans, nas_sorted

def impute_means(df):
    nan_cols = some_nans(df)
    col_means = df[nan_cols].mean()
    imputed_df = df.fillna(col_means)
    return imputed_df

# find columns with nas, impute means

In [39]:
col_means = df[nan_cols].mean()

In [44]:
imputed_df = df.fillna(col_means)

In [45]:
some_nans(imputed_df)

Index([], dtype='object')

In [61]:
nas_sorted(imputed_df)[0:3]

host_acceptance_rate        22552
jurisdiction_names          22552
last_reviewDayofweek_6.0        0
dtype: int64

In [63]:
cleaned_df = imputed_df.drop(columns = ['host_acceptance_rate', 'jurisdiction_names'])

In [64]:
y = cleaned_df.price

In [437]:
X = cleaned_df.drop(columns=['price'])

In [66]:
some_nans(imputed_df)

Index([], dtype='object')

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [69]:
cleaned_df.isnull().values.any()

False

In [126]:
from sklearn import preprocessing

In [439]:
scaled_features = preprocessing.scale(X)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [440]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.33, random_state=42)

### Feature Selection

In [130]:
estimator = LinearRegression()
estimator.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [132]:
coefs = estimator.coef_

In [134]:
cols = X.columns

In [138]:
coefs_cols = np.stack((coefs, cols)).T

In [143]:
coefs_cols.sort(axis = 0)

In [153]:
# coefs_cols[:60]
# coefs_cols[-40:]

In [151]:
import sklearn
p_vals = sklearn.feature_selection.f_regression(X_train, y_train, center=True)[-1]

In [165]:
p_val_cols = np.stack((p_vals, cols)).T

In [168]:
threshold_ps = p_val_cols[p_val_cols[:, 0] < .05]

In [169]:
threshold_ps.sort(axis = 0)

In [172]:
threshold_cols = threshold_ps[:, -1]

In [175]:
top_hundred_X =  X[threshold_cols]

In [176]:
scaled_top_hundred_X = preprocessing.scale(top_hundred_X)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [177]:
X_train_top, X_test_top, y_train, y_test = train_test_split(scaled_top_hundred_X, y, test_size=0.33, random_state=42)

In [178]:

# scaled_features = preprocessing.scale(X)
estimator = LinearRegression()
# estimator.fit(X_train, y_train)
estimator.fit(X_train_top, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [179]:
estimator.score(X_train_top, y_train)

0.16963907886754037

## Using RFE

In [449]:


# 6:38

In [260]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.33, random_state=42)

from sklearn.feature_selection import RFE
selectors = []
for idx in list(range(2, 300, 10))[::-1]:
    estimator = LinearRegression(n_jobs=-1)
    rfe = RFE(estimator, idx, step=1)
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    
    estimator.fit(X_train_rfe,y_train)
    score = estimator.score(X_test_rfe,y_test)
    selectors.append((estimator, score, idx))
selectors

[(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10076530257057671,
  292),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10235048809212934,
  282),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10042859826085493,
  272),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10249942317834858,
  262),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.1017775061397078,
  252),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10210962926812306,
  242),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.1015417985139625,
  232),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10257093328868626,
  222),
 (LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  0.10162415354987853,
  

In [265]:
[idx for selector, score, idx in selectors[18:20]]

[112, 102]

In [262]:
prime_selectors

[212, 202, 192, 182, 172, 162, 152]

In [452]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.33, random_state=35)

In [453]:
#score 112
estimator = LinearRegression(n_jobs=-1)
rfe = RFE(estimator, 112, step=1)
X_train_rfe = rfe.fit_transform(X_train,y_train)
X_test_rfe = rfe.transform(X_test)

estimator.fit(X_train_rfe,y_train)
score = estimator.score(X_test_rfe,y_test)

In [460]:
len(rfe.support_)

303

In [462]:
len(X.columns)

303

In [463]:
unscaled_X_train, unscaled_X_test, unscaled_y_train, unscaled_y_test = train_test_split(X, y, test_size=0.33, random_state=35)


In [464]:
model = LinearRegression()
model.fit(unscaled_X_train, unscaled_y_train)
model.score(unscaled_X_test, unscaled_y_test)

0.12127156186709964

In [454]:
score

0.11440209280770308

In [442]:
features_one_twelve = X.columns[rfe.support_].to_numpy()
coefs = estimator.coef_
abs_coefs = abs(coefs)
paired = np.hstack((features_one_twelve.reshape(-1, 1), coefs.reshape(-1, 1), abs_coefs.reshape(-1, 1)))
ordered_cols = paired[paired[:, -1].argsort()][::-1]
top_limited_cols = ordered_cols[:-42]

In [450]:
X_train.shape

(15109, 303)

In [451]:
type(X_train)

numpy.ndarray

In [448]:
ordered_cols[:, 0]

array(['last_scrapedElapsed', 'last_scrapedDayofyear', 'last_scrapedDay',
       'calendar_last_scrapedDayofyear',
       'calendar_last_scrapedDayofweek_4', 'last_scrapedDayofweek_4',
       'calendar_last_scrapedElapsed', 'calendar_last_scrapedDay',
       'host_picture_url_is_na', 'first_review_is_na',
       'reviews_per_month_is_na', 'host_thumbnail_url_is_na',
       'host_has_profile_pic_is_na', 'host_since_is_na',
       'host_listings_count_is_na', 'host_is_superhost_is_na',
       'host_total_listings_count', 'host_listings_count',
       'host_response_time_other', 'host_response_time_is_na',
       'host_identity_verified_is_na', 'host_name_is_na',
       'host_total_listings_count_is_na',
       'neighbourhood_group_cleansed_Friedrichshain-Kreuzberg',
       'neighbourhood_cleansed_Tempelhofer Vorstadt',
       'neighbourhood_cleansed_Frankfurter Allee Süd FK',
       'neighbourhood_cleansed_Frankfurter Allee Nord',
       'neighbourhood_cleansed_südliche Luisenstadt',
   

### Recombine Features that appear Linear

In [320]:
features_limited_df = X[top_limited_cols]

In [331]:
from_dataset_df = pd.read_csv('listings_summary.csv.zip')

In [333]:
host_since = from_dataset_df.host_since

In [334]:
last_review = from_dataset_df.last_review

In [335]:
from date_lib import add_datepart

In [337]:
add_datepart(from_dataset_df, 'host_since')

In [338]:
add_datepart(from_dataset_df, 'last_review')

In [340]:
last_review_month = from_dataset_df.last_reviewMonth

In [341]:
host_sinceMonth = from_dataset_df.host_sinceMonth

In [421]:
last_reviewMonthcols = features_limited_df.loc[:, features_limited_df.columns.map(lambda x: x.startswith('last_reviewMonth'))].columns

In [422]:
len(last_reviewMonthcols)

11

In [423]:
hostSincecols = X.loc[:, X.columns.map(lambda x: x.startswith('host_sinceMonth'))]

In [424]:
cols_to_remove = list(hostSincecols) +  list(last_reviewMonthcols)

In [425]:
X_with_removed = X.drop(columns=cols_to_remove)

In [426]:
len(X_with_removed.columns)

281

In [427]:
X_with_removed['host_sinceMonth'] = host_sinceMonth

In [428]:
X_with_removed['last_reviewMonth'] = last_review_month

In [429]:
imputed_with_removed = impute_means(X_with_removed)

In [430]:
imputed_with_removed['price'] = cleaned_df.price

In [431]:
imputed_with_removed.to_feather('imputed_with_removed')

In [432]:
scaled_features_with_removed = preprocessing.scale(imputed_with_removed)
X_train, X_test, y_train, y_test = train_test_split(scaled_features_with_removed, y, test_size=0.33, random_state=42)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [433]:
#score 112
estimator = LinearRegression(n_jobs=-1)
rfe = RFE(estimator, 112, step=1)
X_train_rfe = rfe.fit_transform(X_train,y_train)
X_test_rfe = rfe.transform(X_test)

estimator.fit(X_train_rfe,y_train)
score = estimator.score(X_test_rfe,y_test)

def feature_importances(df, estimator):
    df_cols = df.columns[rfe.support_].to_numpy()
    coefs = estimator.coef_
    abs_coefs = abs(coefs)
    paired = np.hstack((df_cols.reshape(-1, 1), coefs.reshape(-1, 1), abs_coefs.reshape(-1, 1)))
    ordered_cols = paired[paired[:, -1].argsort()][::-1]
    return ordered_cols

In [434]:
def feature_importances(df, estimator):
    df_cols = df.columns[rfe.support_].to_numpy()
    coefs = estimator.coef_
    abs_coefs = abs(coefs)
    paired = np.hstack((df_cols.reshape(-1, 1), coefs.reshape(-1, 1), abs_coefs.reshape(-1, 1)))
    ordered_cols = paired[paired[:, -1].argsort()][::-1]
    return ordered_cols

In [435]:
importances = feature_importances(imputed_with_removed,estimator)

In [436]:
importances

array([['price', 220.2613265910614, 220.2613265910614],
       ['host_identity_verified_is_na', -13.27420077739558,
        13.27420077739558],
       ['host_has_profile_pic_is_na', -13.144912210070528,
        13.144912210070528],
       ['host_thumbnail_url_is_na', -10.49713016843962,
        10.49713016843962],
       ['host_listings_count_is_na', 10.42586174177467,
        10.42586174177467],
       ['host_name_is_na', 8.704628490767004, 8.704628490767004],
       ['host_total_listings_count_is_na', 7.919681003546115,
        7.919681003546115],
       ['host_picture_url_is_na', 7.430829168309159, 7.430829168309159],
       ['host_is_superhost_is_na', 5.976394250805642, 5.976394250805642],
       ['host_since_is_na', -3.5411514994905575, 3.5411514994905575],
       ['last_scrapedDay', -1.5122195303008361, 1.5122195303008361],
       ['reviews_per_month_is_na', -1.340751161136195, 1.340751161136195],
       ['first_review_is_na', 1.3407511603155513, 1.3407511603155513],
       ['nei

In [403]:
np.where(importances[:, 0] == 'last_reviewDayofyear')

(array([43]),)

In [411]:
import pandas as pd 

pd.DataFrame(importances[:43]).to_csv("importances.csv")

In [414]:
# pd.read_csv('importances.csv').to_numpy()[:, 1:]

### From here can dedicate feature engineering

### Resources

* Feature selection in theory
    * Want those that are correlated, or pearson correlated with the outputs
        * Could compare both techniques

In [ ]:
https://towardsdatascience.com/why-how-and-when-to-apply-feature-selection-e9c69adfabf2